# Serialize Trained Hybrid Keras/TF Model for Tensorflow Serving

In [1]:
from __future__ import division, print_function
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy
import os
import shutil
import tensorflow as tf

In [2]:
DATA_DIR = "../../data"
TEST_FILE = os.path.join(DATA_DIR, "mnist_test.csv")

IMG_SIZE = 28
BATCH_SIZE = 128
NUM_CLASSES = 10

MODEL_DATA_DIR = "../../data/01-tf-serving"
MODEL_EPOCH = 5
MODEL_NAME = os.path.join(MODEL_DATA_DIR, "model-" + str(MODEL_EPOCH))

EXPORT_DIR_ROOT = os.path.join(DATA_DIR, "tf-export")
EXPORT_MODEL_NAME = "mnist_cnn"
EXPORT_MODEL_VERSION = 1

## Generate data for testing

In [3]:
def parse_file(filename):
    xdata, ydata = [], []
    fin = open(filename, "rb")
    i = 0
    for line in fin:
        if i % 10000 == 0:
            print("{:s}: {:d} lines read".format(
                os.path.basename(filename), i))
        cols = line.strip().split(",")
        ydata.append(int(cols[0]))
        xdata.append(numpy.reshape(numpy.array([float(x) / 255. 
            for x in cols[1:]]), (IMG_SIZE, IMG_SIZE, 1)))
        i += 1
    fin.close()
    print("{:s}: {:d} lines read".format(os.path.basename(filename), i))
    y = numpy.array(ydata)
    X = numpy.array(xdata)
    return X, y

Xtest, ytest = parse_file(TEST_FILE)
print(Xtest.shape, ytest.shape)

mnist_test.csv: 0 lines read
mnist_test.csv: 10000 lines read
(10000, 28, 28, 1) (10000,)


In [4]:
def datagen(X, y, batch_size=BATCH_SIZE, num_classes=NUM_CLASSES):
    ohe = OneHotEncoder(n_values=num_classes)
    while True:
        shuffled_indices = numpy.random.permutation(numpy.arange(len(y)))
        num_batches = len(y) // batch_size
        for bid in range(num_batches):
            batch_indices = shuffled_indices[bid*batch_size:(bid+1)*batch_size]
            Xbatch = numpy.zeros((batch_size, X.shape[1], X.shape[2], X.shape[3]), 
                                 dtype="float32")
            Ybatch = numpy.zeros((batch_size, num_classes), dtype="int32")
            for i in range(batch_size):
                Xbatch[i] = X[batch_indices[i]]
                Ybatch[i] = ohe.fit_transform(y[batch_indices[i]]).todense()
            yield Xbatch, Ybatch

self_test_gen = datagen(Xtest, ytest)
Xbatch, Ybatch = self_test_gen.next()
print(Xbatch.shape, Xbatch.dtype, Ybatch.shape, Ybatch.dtype)

(128, 28, 28, 1) float32 (128, 10) int32


## Restore trained model

This [blog post](http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/) contains lots of good information on how to save and restore Tensorflow models.

In [5]:
# sess = tf.Session()
# tf.contrib.keras.backend.set_session(sess)
tf.contrib.keras.backend.set_learning_phase(0)
sess = tf.contrib.keras.backend.get_session()
with sess.as_default():
    saver = tf.train.import_meta_graph(MODEL_NAME + ".meta")
    saver.restore(sess, tf.train.latest_checkpoint(MODEL_DATA_DIR))

INFO:tensorflow:Restoring parameters from ../../data/01-tf-serving/model-5


## Evaluate Model with test data

This is to make sure that the restored model is the same as the one we trained.

In [6]:
ys, ys_ = [], []
test_gen = datagen(Xtest, ytest, BATCH_SIZE)
with sess.as_default():
    graph = tf.get_default_graph()
    X = graph.get_tensor_by_name("data/X:0")
#     Y = graph.get_tensor_by_name("data/Y:0")
    Y_ = graph.get_tensor_by_name("sequential_1/dense_2/Softmax:0")
    learning_phase = graph.get_tensor_by_name("dropout/keras_learning_phase:0")
#     tf.contrib.keras.backend.set_learning_phase(0)
    num_batches = len(Xtest) // BATCH_SIZE
    for _ in range(num_batches):
        Xbatch, Ybatch = test_gen.next()
        Ybatch_ = sess.run(Y_, feed_dict={X: Xbatch, learning_phase: 0})
        ys.extend(numpy.argmax(Ybatch, axis=1))
        ys_.extend(numpy.argmax(Ybatch_, axis=1))

acc = accuracy_score(ys_, ys)
cm = confusion_matrix(ys_, ys)
print("Accuracy: {:.4f}".format(acc))
print("Confusion Matrix")
print(cm)

Accuracy: 0.9898
Confusion Matrix
[[ 974    0    1    0    0    2    6    1    5    1]
 [   1 1131    2    0    0    0    2    1    1    3]
 [   0    1 1022    2    0    0    0    3    2    0]
 [   0    1    0 1002    0    8    0    1    1    0]
 [   0    0    1    0  971    0    1    0    0    4]
 [   0    1    0    2    0  878    5    0    0    5]
 [   1    1    0    0    3    1  940    0    0    0]
 [   0    0    5    1    0    0    0 1019    5    3]
 [   0    0    0    1    1    1    1    1  955    2]
 [   1    0    0    0    6    0    0    0    4  990]]


## Export Model to form suitable by TF-Serving

TF-Serving needs its models to be serialized to the [SavedModel format](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md). The following code is largely adapted from the [mnist_saved_model.py](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_saved_model.py). More information on this [Tensorflow Serving documentation page](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/serving_basic.md).

The resulting exported model directory structure is as follows:

    .
    └── mnist_cnn_model
        └── 1
            ├── saved_model.pb
            └── variables
                ├── variables.data-00000-of-00001
                └── variables.index

Here /tmp/ corresponds to a top level `EXPORT_DIR_ROOT`, which contains the `EXPORT_MODEL_NAME_model` subdirectory. Within this is another subdirectory corresponding to the `EXPORT_MODEL_VER` which contains the protobuf file saved_model.pb containing the metadata file for the graph. The variables subdirectory contains the variables index and data files respectively.

In [7]:
serialized_tf_example = tf.placeholder(tf.string, name="tf_example")

In [9]:
export_dir = os.path.join(EXPORT_DIR_ROOT, EXPORT_MODEL_NAME + "_model")
shutil.rmtree(EXPORT_DIR_ROOT, True)
export_path = os.path.join(export_dir, str(EXPORT_MODEL_VERSION))
print("Exporting model to {:s}".format(export_path))

Exporting model to ../../data/tf-export/mnist_cnn_model/1


In [10]:
builder = tf.saved_model.builder.SavedModelBuilder(export_path)

In [12]:
tensor_info_x = tf.saved_model.utils.build_tensor_info(X)
tensor_info_y = tf.saved_model.utils.build_tensor_info(Y_)

prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={"images": tensor_info_x},
        outputs={"scores": tensor_info_y},
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))
print(prediction_signature)

inputs {
  key: "images"
  value {
    name: "data/X:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
      dim {
        size: 1
      }
    }
  }
}
outputs {
  key: "scores"
  value {
    name: "sequential_1/dense_2/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 10
      }
    }
  }
}
method_name: "tensorflow/serving/predict"



In [13]:
legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
builder.add_meta_graph_and_variables(
    sess, [tf.saved_model.tag_constants.SERVING],
    signature_def_map = {
        "predict": prediction_signature,
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
            prediction_signature
    },
    legacy_init_op=legacy_init_op)
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../../data/tf-export/mnist_cnn_model/1/saved_model.pb


'../../data/tf-export/mnist_cnn_model/1/saved_model.pb'